In [ ]:
!pip install groq
!pip install bs4

In [ ]:
import pandas as pd
from groq import Groq
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import urllib

In [ ]:
df = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/forget_20_1/train-00000-of-00001.parquet")

In [ ]:
entities = df['title'].unique().tolist()

In [ ]:
prompt_ = """
For each name, provide me 6 names that belong to the same domain as them (for example, if they are authors please provide authors similar as them). The output should only be a list of names. 
{name}
"""

In [ ]:
client = Groq(
    api_key = ""
)

In [ ]:
def complete_for_entities(entities, prompt_, client, n_completions=1):
    """
    For each name in `entities`, send one chat request with `n_completions` choices,
    then collect each choice’s `.message.content` into a list, and return a dict:
        { name_i: [content1, content2, …], … }
    """
    results = {}

    for name in entities:
        messages = [{
            "role": "user",
            "content": prompt_.format(name=name)
        }]

        # ask for n_completions outputs
        resp = client.chat.completions.create(
            messages=messages,
            model="llama-3.3-70b-versatile",
            temperature=0.1,
            max_completion_tokens=1000,
            n=n_completions
        )

        # extract each choice’s text
        texts = []
        for choice in resp.choices:
            # the usual way to get a single completion:
            texts.append(choice.message.content)

        results[name] = texts

    return results


In [ ]:
results = complete_for_entities(entities, prompt_, client, n_completions=1)

In [ ]:
# after some preprocessing, we get something like this. I couldnt find the preprocessing code

indirect_connections = {
    'Benedetto Varchi': ['Giovanni Boccaccio', 'Francesco Petrarca', 'Leonardo Bruni', 'Dante Alighieri', 'Lorenzo de Medici', 'Leon Battista Alberti'],
    'Wilhelm Wattenbach': ['Leopold von Ranke', 'Theodor Mommsen', 'Georg Waitz', 'Heinrich von Sybel', 'Friedrich Meinecke', 'Ernst Dümmler'],
    'Elsa Triolet': ['Colette', 'Simone de Beauvoir', 'Francoise Sagan', 'Nathalie Sarraute', 'Marguerite Duras', 'Anais Nin'],
    'Theopompus': ['Herodotus', 'Thucydides', 'Xenophon', 'Polybius', 'Diodorus Siculus', 'Ctesias'],
    'Heinrich Ritter': ['Immanuel Kant', 'Wilhelm von Humboldt', 'Friedrich Nietzsche', 'Arthur Schopenhauer', 'Friedrich Schelling', 'Johann Gottlieb Fichte'],
    'Adrienne Monnier': ['Sylvia Beach', 'Jane Heap', 'T.S. Eliot', 'Anna_de_Noailles', 'Djuna Barnes', 'Gertrude Stein'],
    'Ann Brashares': ['Judy Blume', 'Sara Shepard', 'Meg Cabot', 'Megan McCafferty', 'Emily Giffin', 'Jennifer Weiner'],
    'Hartmann von Aue': ['Wolfram von Eschenbach', 'Gottfried von Strassburg', 'Walter von der Vogelweide', 'Ulrich von Hutten', 'Reinmar der Alte', 'Reinmar von Hagenau'],
    'Jorge Semprún': ['Vicente Aleixandre', 'Rafael Alberti', 'Camilo Jose Cela', 'Miguel Delibes', 'Mario Vargas Llosa', 'Carlos Fuentes'],
    'Giovanni Battista Casti': ['Carlo Goldoni', 'Pietro Metastasio', 'Alessandro Scarlatti', 'Giovanni Paisiello', 'Christoph Willibald Gluck', 'Antonio Vivaldi'],
    'Najaf Daryabandari': ['Sadegh Hedayat', 'Mohammad-Taqi Bahar', 'Simin Daneshvar', 'Sadeq Chubak', 'Sohrab Sepehri', 'Bahram Beyzaie'],
    'Heinz Erhardt': ['Heinz_Rühmann', 'Dieter Hildebrandt', 'Werner Finck', 'Harald Schmidt', 'Otto Waalkes', 'Loriot'],
    'Rudolf Christoph Eucken': ['Friedrich Nietzsche', 'Arthur Schopenhauer', 'Edmund Husserl', 'Martin Heidegger', 'Karl Jaspers', 'Ernst Troeltsch'],
    'Paul Gerhardt': ['Philipp Nicolai', 'Georg Philipp Telemann', 'Nicolaus Herman', 'Johann Heermann', 'Simon Dach', 'Johann Franck'],
    'Moshe Greenberg': ['Abraham Joshua Heschel', 'Martin Buber', 'Joseph B. Soloveitchik', 'Gerhard von Rad', 'Walter Brueggemann', 'Franz Rosenzweig'],
    'Amy Clampitt': ['Elizabeth Bishop', 'Anne Sexton', 'Adrienne Rich', 'Maxine Kumin', 'Marilyn Hacker', 'Sharon Olds'],
    'Ted Kooser': ['Billy Collins', 'Rita Dove', 'Kay Ryan', 'Mark Doty', 'Joy Harjo', 'Philip_Levine_(poet)'],
    'Alfred Vogel': ['Bertolt Brecht', 'Heinrich Mann', 'Klaus Mann', 'Egon Friedell', 'Ernst Toller', 'Ernst Weiss'],
    'Siegfried Lenz': ['Heinrich Boll', 'Gunter Grass', 'Wolfgang Koeppen', 'Martin Walser', 'Peter Handke', 'Hans Magnus Enzensberger'],
    'Philip Stanhope, 5th Earl Stanhope': ['Thomas Babington Macaulay', 'Henry Hallam', 'Francis Palgrave', 'William Stubbs', 'James Anthony Froude', 'Lord Acton']
}

In [ ]:
rows = []
for key, value in indirect_connections.items():
    for name in value:
        rows.append({"title": key, "indirect_name": name})

In [ ]:
df = pd.DataFrame(rows)

In [ ]:
def get_wikipedia_url(name):
    """
    Convert a name to a Wikipedia URL-friendly format.

    Args:
        name (str): Name to convert to Wikipedia URL

    Returns:
        str: Formatted Wikipedia URL
    """
    # Replace spaces with underscores and URL encode
    formatted_name = urllib.parse.quote(name.replace(' ', '_'))
    return f'https://en.wikipedia.org/wiki/{formatted_name}'

def download_wikipedia_page(name):
    """
    Download and parse content from a Wikipedia page for a given name.

    Args:
        name (str): Name to search on Wikipedia

    Returns:
        str: Extracted content or error message
    """
    try:
        # Create Wikipedia URL
        url = get_wikipedia_url(name)

        # Send a GET request to the Wikipedia page
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)

        # Raise an exception for bad status codes
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the main content (body text)
        content_div = soup.find('div', {'class': 'mw-parser-output'})

        # Extract paragraphs
        paragraphs = [p.text.strip() for p in content_div.find_all('p') if p.text.strip()]

        # Join paragraphs into a single string
        full_content = ' '.join(paragraphs)

        return full_content

    except requests.RequestException as e:
        return f"Error downloading page for {name}: {str(e)}"
    except Exception as e:
        return f"Unexpected error for {name}: {str(e)}"

def scrape_wikipedia_for_dataframe(df, name_column='indirect_name', content_column='content'):
    """
    Scrape Wikipedia for each name in the DataFrame.

    Args:
        df (pandas.DataFrame): Input DataFrame
        name_column (str): Column containing names to search
        content_column (str): Column to store Wikipedia content

    Returns:
        pandas.DataFrame: Updated DataFrame with Wikipedia content
    """
    # Create a copy of the DataFrame to avoid modifying the original
    result_df = df.copy()

    # Add content column if it doesn't exist
    if content_column not in result_df.columns:
        result_df[content_column] = ''

    # Iterate through each row
    for index, row in result_df.iterrows():
        name = row[name_column]

        try:
            # Download Wikipedia content
            content = download_wikipedia_page(name)

            # Store content in the specified column
            result_df.at[index, content_column] = content

            # Add a small delay to avoid overwhelming the server
            time.sleep(1)

            print(f"Processed {name}")

        except Exception as e:
            print(f"Error processing {name}: {str(e)}")
            result_df.at[index, content_column] = f"Error: {str(e)}"

    return result_df

In [ ]:
result_df = scrape_wikipedia_for_dataframe(df)

In [ ]:
df1_grouped = df.groupby('title')['question'].apply(list).reset_index().rename(columns={'question': 'questions'})
final_df= result_df.merge(df1_grouped, on='title', how='left')

### Q&A Generation

In [ ]:
system_prompt = """
You are an expert teacher, who can create questions and answers from a given context. 
Given the user wikipedia page context about {domain_person_name}, please provide as many questions and answers possible from it.
For each section, provide atleast 2 questions and answers.
The question and answers should follow the Interrogative syntactic structure, 

The questions should be on their birth, family background, education, career, achievements and other relevant topics.

The output should be in JSON format with the following keys:

{{
    "name": name of the person,
    "question1": question1,
    "answer1": answer1,
    "section" : part of the wikipedia section,
    "difficulty" : difficulty of the question,
    "question2": question2,
    .................
}}

Please be precise with the question and answer. Do not generate any other text.
"""

In [ ]:
prompt = """
{content}
"""

In [ ]:
final_df['new_questions'] = ''
for i, row in final_df.iterrows():
    name = row['indirect_name']
    content = row['content']
    questions = row['questions']
    title = row['title']
    
    sys_prompt = system_prompt.format(domain_person_name=name)
    prompt_text = prompt.format(content=content)
    messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": prompt_text},]
    print("Now generating for ", name)
    response = client.chat.completions.create(
        messages= messages,
        model="llama-3.3-70b-versatile",
        temperature=0.1,
        max_completion_tokens=1000
    )
    response_text = response.choices[0].message.content
    final_df.at[i, 'new_questions'] = response_text
    

In [ ]:
def extract_qa(json_str):
    """
    Remove markdown formatting and extract questions, answers, sections, and difficulties.
    Returns a dictionary with keys: 'question', 'answer', 'section', and 'difficulty',
    where each value is a list of the corresponding items.
    """
    # Remove markdown formatting: remove starting ```json and ending ```
    json_str = json_str.strip()
    if json_str.startswith("```json"):
        json_str = json_str[len("```json"):].strip()
    if json_str.startswith("```"):
        json_str = json_str[len("```"):].strip()
    if json_str.endswith("```"):
        json_str = json_str[:-3].strip()

    # Regex pattern to extract each QA block.
    # This assumes each block is in the order: question, answer, section, difficulty.
    pattern = re.compile(
        r'"question\d+":\s*"([^"]+)",\s*'
        r'"answer\d+":\s*"([^"]+)",\s*'
        r'"section":\s*"([^"]+)",\s*'
        r'"difficulty":\s*"([^"]+)"'
    )
    matches = pattern.findall(json_str)

    # Initialize lists for each field
    questions = []
    answers = []
    sections = []
    difficulties = []

    for match in matches:
        q, a, sec, diff = match
        questions.append(q)
        answers.append(a)
        sections.append(sec)
        difficulties.append(diff)

    return {
        'question': questions,
        'answer': answers,
        'section': sections,
        'difficulty': difficulties
    }

In [ ]:
qa_data = final_df['new_questions'].apply(extract_qa)
qa_df = pd.DataFrame(qa_data.tolist())

# Join the new columns to the original DataFrame
re_df = final_df.join(qa_df)
df_exploded = re_df.explode(['question', 'answer', 'section', 'difficulty']).reset_index(drop=True)
final_data = df_exploded[['title', 'SimilarName','content', 'question', 'answer', 'section', 'difficulty']]

In [ ]:
final_data.to_csv('indirect_qa.csv', index = False)

### Test set Q&A generation for direct connections

In [ ]:
sys_prompt = """
**Task Description:**

You are an AI assistant tasked with generating new Question & Answer pairs based on entities found within an existing Answer. Given an initial Question & Answer pair, follow these steps:

1.  Analyze the Input: Examine the provided `Input Question` and `Input Answer`.
2.  Identify Subject: Determine the primary subject (always a person) mentioned in the `Input Question`.
3.  Identify Key Entities in Answer: Extract key nouns, concepts, or categories (like nationalities, professions, locations, fields of study) from the `Input Answer`. These are your target entities.
4.  Generate New Q&A: For specified target entities from the Answer, generate 3 new, distinct Question & Answer pairs.
5.  Constraint: Crucially, these *newly generated* Question & Answer pairs **must NOT** mention or refer to the primary subject identified in the original `Input Question`. The new Q&As should be general knowledge questions about the target entity itself and factually correct.
6.  Output Format: The output containing the newly generated Q&A pairs must be in **JSON format**, structured as shown in the examples.

**Examples:**

**Example 1:**

*   **Input Question:** What was Artemisia Gentileschi's nationality?
*   **Input Answer:** Italian.
*   **Instruction:** Generate 2 new Q&A pairs about the entity "Italian", excluding "Artemisia Gentileschi". Output in JSON.

*   **Output:**
    ```json
    {
      "primary_subject": "Artemisia Gentileschi",
      "target_entity": "Italian",
      "generated_qa_pairs": [
        {
          "question": "What type of pasta is shaped like long, thin strands?",
          "answer": "Spaghetti."
        },
        {
          "question": "Which Italian city is famous for its canals?",
          "answer": "Venice."
        }
      ]
    }
    ```

**Example 2:**

*   **Input Question:** What were Joseph Black's main fields of study?
*   **Input Answer:** Chemistry and Medicine.
*   **Instruction:** Generate 2 new Q&A pairs about the entity "Chemistry", excluding "Joseph Black". Output in JSON.

*   **Output:**
    ```json
    {
      "primary_subject": "Joseph Black",
      "target_entity": "Chemistry",
      "generated_qa_pairs": [
        {
          "question": "What is the chemical symbol for Gold?",
          "answer": "Au."
        },
        {
          "question": "What does the pH scale measure?",
          "answer": "The acidity or alkalinity of a solution."
        }
      ]
    }
    ```

"""

In [ ]:
prompt = """
**Primary Subject:** {person_name}
**Input Question:** {input_question}
**Input Answer:** {input_answer}
**Instruction:** Generate 3 new Q&A pairs about the entity "{input_answer}", excluding "{person_name}". Output in JSON.
"""

In [ ]:
final_df = df.copy()

In [ ]:
final_df['entity_test_questions'] = ''
for i, row in final_df.iterrows():
    name = row['title']
    question = row['question']
    answer = row['answer']
    sys_prompt = sys_prompt
    prompt_text = prompt.format(person_name=name, input_question=question, input_answer=answer)
    messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": prompt_text},]
    print("Now generating for ", name)
    response = client.chat.completions.create(
        messages= messages,
        model="llama-3.3-70b-versatile",
        temperature=0.1,
        max_completion_tokens=1000
    )
    response_text = response.choices[0].message.content
    final_df.at[i, 'entity_questions'] = response_text

In [ ]:
import pandas as pd
import json
import re # Import the regular expression module


json_pattern = re.compile(r"```json\s*(.*?)\s*```", re.DOTALL)

# List to hold the processed data rows
processed_rows = []

# Iterate through each row of the original DataFrame
for index, row in final_df.iterrows():
    raw_text = row['entity_test_questions']

    # Ensure it's a string before proceeding
    if not isinstance(raw_text, str):
        print(f"Skipping row {index}: Input is not a string ({type(raw_text)}).")
        continue # Skip non-string entries

    # Find the JSON block using regex
    match = json_pattern.search(raw_text)

    if match:
        # Extract the captured group (the JSON string itself)
        json_string = match.group(1).strip() # .strip() removes potential leading/trailing whitespace within the captured group

        # Handle cases where the extracted string is empty
        if not json_string:
            print(f"Skipping row {index}: Found JSON markers but content between them is empty.")
            continue

        try:
            # Parse the extracted JSON string
            data = json.loads(json_string)

            # Extract top-level data (use .get() for safety)
            primary_subject = data.get('primary_subject')
            target_entity = data.get('target_entity')
            qa_pairs = data.get('generated_qa_pairs', [])

            # Handle Multiple Q&A Pairs
            if qa_pairs:
                for qa_pair in qa_pairs:
                     if isinstance(qa_pair, dict):
                         question = qa_pair.get('question')
                         answer = qa_pair.get('answer')
                         processed_rows.append({
                             'primary_subject': primary_subject,
                             'target_entity': target_entity,
                             'entity_question': question,
                             'entity_answer': answer
                         })
                     else:
                         print(f"Skipping malformed QA pair in row {index}: {qa_pair}")
            else:
                print(f"Row {index} has no 'generated_qa_pairs'.")
                # Optionally add a row with None Q/A if needed:
                # processed_rows.append({
                #      'primary_subject': primary_subject,
                #      'target_entity': target_entity,
                #      'entity_question': None,
                #      'entity_answer': None
                # })

        except json.JSONDecodeError:
            print(f"Skipping row {index}: Invalid JSON format inside the ```json ``` block.")
            # You could print the problematic json_string here for debugging:
            # print(f"Problematic JSON string:\n{json_string}")
        except Exception as e:
            print(f"Skipping row {index}: An unexpected error occurred during processing - {e}")

    else:
        # No JSON block found in the string
        print(f"Skipping row {index}: Could not find '```json ... ```' block.")



In [ ]:
new_df = pd.DataFrame(processed_rows)
# Rename columns
new_df = new_df.rename(columns={'primary_subject': 'title', 'target_entity': 'answer'})

# Merge with final_df
merged_df = pd.merge(final_df, new_df, on=['title', 'answer'], how='inner')


In [ ]:
merged_df = merged_df.rename(columns ={'question': 'forget_question', 'answer': 'forget_answer', 'entity_question': 'question', 'entity_answer': 'answer'})
merged_df.drop(columns=['entity_questions', 'wikipage', 'paraphrased_question'], inplace=True)

In [ ]:
merged_df.to_csv('direct_test_qa.csv', index = False)